In [ ]:
import pandas as pd
import joblib
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
file_url = '../Dataset/phpMawTba.csv'

In [ ]:
df = pd.read_csv(file_url)

In [ ]:
df.head()

In [ ]:
y = df.pop('class')

In [ ]:
cat_columns = [col for col in df.columns if df[col].dtype == 'object']
cat_columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=8)

In [ ]:
column_categories = {}

In [ ]:
for col in cat_columns:
  column_categories[col] = X_train[col].astype('category').cat.categories

In [ ]:
pickle.dump(column_categories, open("categories_data.pkl", "wb"))
pickle.dump(cat_columns, open("categorical_columns.pkl", "wb"))

In [ ]:
def apply_categories(input_df, cat_dict):
  from pandas.api.types import CategoricalDtype

  for col, cat in cat_dict.items():
    input_df[col] = input_df[col].astype(CategoricalDtype(categories=cat))

  return input_df

In [ ]:
X_train_cat = apply_categories(X_train, column_categories)
X_train_cat.dtypes

In [ ]:
X_train_final = pd.get_dummies(X_train_cat, columns=cat_columns)

In [ ]:
rf_model = RandomForestClassifier(random_state=8)
rf_model.fit(X_train_final, y_train)
joblib.dump(rf_model, "model.pkl") 

In [ ]:
import socket
import threading
import requests
import json
from flask import Flask, jsonify, request
import numpy as np

In [ ]:
app = Flask(__name__)

In [ ]:
trained_model = joblib.load("model.pkl")
var_means = pickle.load(open("categories_data.pkl", "rb"))
cat_cols = pickle.load(open("categorical_columns.pkl", "rb"))

In [ ]:
@app.route('/api', methods=['POST'])
def predict():
  data = request.get_json()
  df_test = pd.DataFrame(data, index=[0])
  df_test_clean = apply_categories(df_test, var_means)
  df_test_final = pd.get_dummies(df_test_clean, columns=cat_cols)
  prediction = trained_model.predict(df_test_final)
  str_pred = np.array2string(prediction)
  return jsonify(str_pred)

In [ ]:
flask_thread = threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':80})
flask_thread.start()

In [ ]:
record = X_test.iloc[0,].to_json()
record

In [ ]:
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
ip_address = socket.gethostbyname(socket.gethostname())

In [ ]:
r = requests.post(f"http://{ip_address}/api", data=record, headers=headers)
r.text